# Tutorial 1 - basics of DLM

## Intro

This Jupyter notebook demonstrates basic functionalities of the dynamic_lifetime_model (DLM) framework. 

The notebook is divided into three parts:
1. Imports and basic setup - includes Python imports and basic variable definition;
2. Comparing lifetime functions - visualizes the differences between the hazard function, being the basis of DLM, and the corresponding survival function and probability function;
3. Creating simple models with 'nature' and 'nurture' interventions - demonstrates the changes introduced in a simple system through 'nature' and 'nurture' interventions, i.e., cohort and period effects, respectively. 

More details can be found in the publication:
> Krych, K., Müller, DB. & Pettersen, JB. (2024). The ‘nature’ and ‘nurture’ of product lifetimes in dynamic stock modeling. Journal of Industrial Ecology.  

## Imports and basic setup


The average lifetime $\mu$ is described by equation $\mu = \lambda \Gamma (1+1/k) $, where $\lambda$ is the scale parameter, $\Gamma$ is the gamma function and $k$ is the shape parameter. Consequently, if we keep the shape parameter constant, then an increase of 10% in average lifetime corresponds to an increase of 10% in the scale parameter. 

In [ ]:
import sys, os
import numpy as np
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir, 'ODYM/odym/modules')))
import dynamic_stock_model as dsm
sys.path.append(os.getcwd() + '/..')
import dynamic_lifetime_model as dlm
import matplotlib.pyplot as plt

In [ ]:
# time vector
t = np.arange(1950,2050+1)
stock = 10000 / (1 + np.exp(-0.15 * (t - 2010))) # logistic stock saturating at 10000 units

# lifetime data
scale = 10 # scale parameter of the Weibull distribution (often denoted as lambda)
shape = 2 # shape parameter of the Weibull distribution (often denoted as k)
lt_ext = 0.2
effect_year = 2026

export_figs_to_pdf = True
export_figs_to_xlsx = True

In [ ]:
# function used to save data in Excel files without deleting other sheets (for Supplementary Information)
def df_to_excel_SI(excel, df, sheet_name):
    writer = pd.ExcelWriter(excel, mode='a', if_sheet_exists="overlay", engine='openpyxl') 
    df.to_excel(writer, sheet_name=sheet_name, index=True,header=True)
    writer.close()

excel_SI = 'SI_The_nature_and_nurture_of_product_lifetimes.xlsx'

## Comparing lifetime functions

The basic assumption of DLM is that all the stock and flow calculations are based on the hazard function. Below, we see three different lifetime functions (the probability function, survival function, and the hazard function) plotted for the Weibull distributions $W(t,10,2)$ and $W(t,10 \cdot 120\%,2)$ and a third distribution, where the lifetime is increased from $W(t,10,2)$ by 20% after the 10th year. We see that the third distribution results in values that overlap distribution 1 and 2 only in the case of the hazard function. 

In [ ]:
# calculate hazard functions
hz_1 = dlm.compute_hz_from_lt_par(t,{'Type': 'Weibull', 'Shape': dlm.create_2Darray(t,shape), 'Scale': dlm.create_2Darray(t,scale)})[:,0]
hz_2 = dlm.compute_hz_from_lt_par(t,{'Type': 'Weibull', 'Shape': dlm.create_2Darray(t,shape), 'Scale': dlm.create_2Darray(t,scale*(1+lt_ext))})[:,0]
hz_12 = np.concatenate((hz_1[:11], hz_2[11:])) #switch to hz_2 after the 10th year

# calculate survival functions
sf_1 = dlm.compute_sf_from_hz(hz_1)
sf_2 = dlm.compute_sf_from_hz(hz_2)
sf_12 = dlm.compute_sf_from_hz(hz_12)

# calculate probability functions
pdf_1 = dlm.compute_pdf_from_sf(sf_1)
pdf_2 = dlm.compute_pdf_from_sf(sf_2)
pdf_12 = dlm.compute_pdf_from_sf(sf_12)

In [ ]:
fig = plt.figure(figsize=(14, 3))
if export_figs_to_pdf:
    fig.set_dpi(800)
gs = fig.add_gridspec(1, 3,wspace=0.3)
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(np.arange(len(t[70:])), pdf_1[:-70], linestyle='-', linewidth=3, color='tab:orange')
ax1.plot(np.arange(len(t[70:])), pdf_2[:-70], linestyle='-', linewidth=3, color='tab:green')
ax1.scatter(np.arange(len(t[70:])), pdf_12[:-70], color='black', s=8, zorder=3)
ax1.axvline(10, color='gray', linewidth=1, linestyle='--')
ax1.set_xlabel('t')
ax1.set_ylabel('Probability')
ax1.text(-8,ax1.get_ylim()[1],'A', fontsize=15)

ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(np.arange(len(t[70:])), sf_1[:-70], linestyle='-', linewidth=3, color='tab:orange')
ax2.plot(np.arange(len(t[70:])), sf_2[:-70], linestyle='-', linewidth=3, color='tab:green')
ax2.scatter(np.arange(len(t[70:])), sf_12[:-70], color='black', s=8, zorder=3)
ax2.axvline(10, color='gray', linewidth=1, linestyle='--')
ax2.set_xlabel('t')
ax2.set_ylabel('Proportion surviving')
ax2.text(-8,ax2.get_ylim()[1],'B', fontsize=15)

ax3 = fig.add_subplot(gs[0, 2])
ax3.plot(np.arange(len(t[70:])), hz_1[:-70], linestyle='-', linewidth=3, color='tab:orange', label='W(t;10,2) (baseline)', zorder=1)
ax3.plot(np.arange(len(t[70:])), hz_2[:-70], linestyle='-', linewidth=3, color='tab:green', label='W(t;12,2) (20% longer)', zorder=2)
ax3.scatter(np.arange(len(t[70:])), hz_12[:-70], color='black', label='W(t;10,2) extended by 20% after $t=10$', s=8, zorder=3)
ax3.axvline(10, color='gray', linewidth=1, linestyle='--')
ax3.set_xlabel('t')
ax3.set_ylabel('Hazard rate')
ax3.text(-8,ax3.get_ylim()[1],'C', fontsize=15)

fig.legend(bbox_to_anchor=(0.5,-0.02), loc="upper center", fontsize=10)
plt.show()
if export_figs_to_pdf:
    fig.savefig('Fig1.pdf', format='pdf', bbox_inches = "tight")
if export_figs_to_xlsx:
    data = np.concatenate(([pdf_1[:-70]],[pdf_2[:-70]],[pdf_12[:-70]],[sf_1[:-70]],[sf_2[:-70]],[sf_12[:-70]],[hz_1[:-70]],[hz_2[:-70]],[hz_12[:-70]]), axis=0)
    col_names = ['pdf: W(t;10,2)', 'pdf: W(t;12,2)', 'pdf: W(t;10,2) extended', 
                 'sf: W(t;10,2)', 'sf: W(t;12,2)', 'sf: W(t;10,2) extended', 
                 'hz: W(t;10,2)', 'hz: W(t;12,2)', 'hz: W(t;10,2) extended']
    df = pd.DataFrame(data=data.T, index=pd.MultiIndex.from_product([np.arange(len(pdf_1[:-70]))], names=['time']), columns=col_names)
    df_to_excel_SI(excel_SI,df,'Figure 1')

## Creating simple models with 'nature' and 'nurture' interventions

Now, we will create three instances of the DynamicLifetimeModel class: 
- a baseline model with only $W(t;10,2)$, 
- a model with a 'nature' intervention (cohort effect) that increases the lifetime by 20% after 2026, 
- a model with a 'nurture' intervention (period effect) that increases  the lifetime by 20% after 2026. 

The first two models are also created using the dynamic_stock_model (DSM) library from ODYM for comparison. The two libraries handle lifetimes differently: DLM uses the hazard function while DSM uses the survival function. By using the hazard function, DLM can model the "period effects" case, which cannot easily be done in DSM. 

### Modeling using the dynamic_lifetime_model (DLM) library

In [ ]:
# baseline
DLM = dlm.DynamicLifetimeModel(s=stock, t=t)
DLM.lt = {'Type': 'Weibull', 
          'Scale': DLM.create_2Darray(scale), #two-dimensional parameter (time by cohort)
          'Shape': DLM.create_2Darray(shape)  #two-dimensional parameter (time by cohort)
          }
DLM.compute_stock_driven_model()

# nature intervention - cohort effect
DLM_cohort = dlm.DynamicLifetimeModel(s=stock, t=t)
DLM_cohort.lt = {'Type': 'Weibull', 
                'Scale': DLM_cohort.create_2Darray(scale), 
                'Shape': DLM_cohort.create_2Darray(shape)
                }
DLM_cohort.lt['Scale'] = DLM_cohort.add_cohort_effect(DLM_cohort.lt['Scale'],1+lt_ext, effect_year, ref='relative')
DLM_cohort.compute_stock_driven_model()

# nurture intervention - period effect
DLM_period = dlm.DynamicLifetimeModel(s=stock, t=t)
DLM_period.lt = {'Type': 'Weibull', 
                'Scale': DLM_period.create_2Darray(scale), 
                'Shape': DLM_period.create_2Darray(shape)
                }
DLM_period.lt['Scale'] = DLM_period.add_period_effect(DLM_period.lt['Scale'],1+lt_ext, effect_year, ref='relative')
DLM_period.compute_stock_driven_model()

### Modeling using the dynamic_stock_model (DSM) library

Now, we create the equivalents of the first two models in the DSM library and compare the results with DLM. As seen below, the difference between the two is negligible. 

In [ ]:
# baseline
DSM = dsm.DynamicStockModel(s=stock, t=t)
DSM.lt = {'Type': 'Weibull', 
          'Scale': np.repeat(scale,len(t)), #one-dimensional parameter (cohort)
          'Shape': np.repeat(shape,len(t))  #one-dimensional parameters (cohort)
          }
DSM.compute_stock_driven_model()

# nature intervention - cohort effect
DSM_cohort = dsm.DynamicStockModel(s=stock, t=t)
# to reflect the lifetime increase, we multiply the original values by a multiplier of 1 or more
multiplier = np.ones_like(t, dtype=float) 
multiplier[effect_year-t[0]:] = np.repeat(1+lt_ext, len(multiplier[effect_year-t[0]:])) # increase the values after effect_year
DSM_cohort.lt = {'Type': 'Weibull', 
                'Scale': np.repeat(scale,len(t))*multiplier,
                'Shape': np.repeat(shape,len(t))
                }
DSM_cohort.compute_stock_driven_model()

print(f'The relative difference between DLM and DSM is:'
      f'\n - for the "baseline" case: {np.sum(np.abs(DLM.i - DSM.i))/np.sum(DSM.i)}'
      f'\n - for the "cohort effect" case: {np.sum(np.abs(DLM_cohort.i - DSM_cohort.i))/np.sum(DSM_cohort.i)}')

### Results - inflows

In order to see how the different scenarios compare with each other, we plot the results. The year of the intervention is marked by a vertical dashed line. We note that the "nurture" intervention has an immediate effect compared to the "nature" intervention which takes time as it relies on stock replacement. In either case, the effect could be implemented with a transition period (linear or S-shaped/logistic), which would introduce the change gradually, effectively smoothening the inflow curve seen below. 

In [ ]:
plt.figure(figsize=(6, 3))
plt.plot(t,DLM.i, label='Baseline')
plt.plot(t,DLM_cohort.i, label='Cohort effects')
plt.plot(t,DLM_period.i, label='Period effects')
plt.axvline(effect_year, color='gray', linestyle='--')
plt.title('Inflows')
plt.legend()
plt.show()

### Results - age of stock and outflows

We can also demonstrate the changes that happen in the system by plotting the mean age of items in stock. Again, we see that the period effect case gives an immediate result while the cohort effect case lags behind. Note that the age values below are scaled by inflows - if they weren't, we would see more fluctuations in age, e.g., the age before 2026 would not have been stable (it would be driven down by the relatively higher inflows of younger items).

In [ ]:
fig = plt.figure(figsize=(10, 2.5))
gs = fig.add_gridspec(1, 2,wspace=0.2)
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(t,DLM.calculate_age_stock(scale_by_inflow=True))
ax1.plot(t,DLM_cohort.calculate_age_stock(scale_by_inflow=True))
ax1.plot(t,DLM_period.calculate_age_stock(scale_by_inflow=True))
ax1.axvline(effect_year, color='gray', linestyle='--')
ax1.set_xlim(2000,2050)
ax1.set_ylim(0,11.5)
ax1.set_title('Age of stock')

ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(t,DLM.calculate_age_outflow(scale_by_inflow=True), label='Baseline')
ax2.plot(t,DLM_cohort.calculate_age_outflow(scale_by_inflow=True), label='Cohort effects')
ax2.plot(t,DLM_period.calculate_age_outflow(scale_by_inflow=True), label='Period effects')
ax2.axvline(effect_year, color='gray', linestyle='--')
ax2.set_xlim(2000,2050)
ax2.set_ylim(0,11.5)
ax2.set_title('Age of outflows')

fig.legend(bbox_to_anchor=(0.5,-0.02), loc="upper center", fontsize=10)
plt.show()

### Results - hazard matrix

Finally, we can visualize the hazard rate for each time period and cohort. First, we see the hazard matrix for the baseline case, where a clear diagonal pattern indicates that age is the only factor influencing the probability of product discard. In the following plot, we see that the cohort effect introduces a vertical disturbance to the matrix, while and the period effect introduces a horizontal disturbance. 

In [ ]:
# find the maximum hazard rate among all three models
vmax = max(np.max(DLM.hz[40:,40:]), np.max(DLM_cohort.hz[40:,40:]), np.max(DLM_period.hz[40:,40:]))
for interval in [0.2, 0.1,0.05, 0.025]:
    if vmax/interval > 3 and vmax/interval < 8:
        ticks = list(np.arange(0,vmax,interval))
        break

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
if export_figs_to_pdf:
    fig.set_dpi(800)
matrix = DLM.hz[40:,40:]
mask = np.zeros_like(matrix)
mask[np.triu_indices_from(mask)] = True
plot_matrix_mask = np.ma.masked_array(matrix, mask=mask)
img = ax.imshow(plot_matrix_mask,cmap='Oranges',vmax=vmax)
ax.set_xticks(ticks=np.arange(0, t[-1]-t[40]+1, 20), labels=t[40::20], rotation=90)
ax.set_yticks(ticks=np.arange(0, t[-1]-t[40]+1, 20), labels=t[40::20])
ax.xaxis.set_ticks_position('top')
ax.xaxis.set_label_position('top')
ax.set_ylabel('Time', fontsize=10)
ax.set_xlabel('Cohort', fontsize=10)

cbar = plt.colorbar(img, ticks=ticks)
cbar.ax.get_yaxis().labelpad = 8
cbar.ax.set_ylabel('Hazard rate')
 
ax.text(10,40,'Age', fontsize=10, rotation=45)
x = np.flip(np.arange(0,60+1))/2
y = 60-x
ax.plot((0,30), (60,30), lw=1, color='black')
for i in np.arange(60)[::10]:
    ax.plot((x[i],x[i]-1), (y[i],y[i]-1), lw=1, color='black')
    ax.text(x[i]-4,y[i]-1.5, i, fontsize=10, rotation=45)
# ax.plot((0,50),(10,60))
plt.tight_layout()
if export_figs_to_pdf:
    fig.savefig('Fig2.pdf', format='pdf', bbox_inches = "tight")
if export_figs_to_xlsx:
    data = DLM.hz[40:,40:]
    df = pd.DataFrame(data=data, index=pd.MultiIndex.from_product([t[40:]], names=['time']), columns=t[40:])
    df_to_excel_SI(excel_SI,df,'Figure 2')

In [ ]:
fig = plt.figure(figsize=(10, 4))
if export_figs_to_pdf:
    fig.set_dpi(800)
gs = fig.add_gridspec(1, 2,wspace=0.5)
ax1 = fig.add_subplot(gs[0, 0])
ax1.text(-18,-10,'A', fontsize=15)
matrix = DLM_cohort.hz[40:, 40:]# -DLM.hz[40:, 40:]
mask = np.zeros_like(matrix)
mask[np.triu_indices_from(mask)] = True
plot_matrix_mask = np.ma.masked_array(matrix, mask=mask)
# extent = t[0], t[-1], t[-1], t[0]
img1 = ax1.imshow(plot_matrix_mask,cmap='Oranges' ,vmax=vmax)
ax1.set_xticks(ticks=np.arange(0, t[-1]-t[40]+1, 20), labels=t[40::20], rotation=90)
ax1.set_yticks(ticks=np.arange(0, t[-1]-t[40]+1, 20), labels=t[40::20])
ax1.set_xlabel('Cohort')
ax1.set_ylabel('Time')
ax1.xaxis.set_ticks_position('top')
ax1.xaxis.set_label_position('top')
ax1.annotate(text='',xy=(36,35),xytext=(36,28),arrowprops=dict(facecolor='black', width=1.2,headwidth=6,headlength=10))
ax1.text(36,28-1,'Cohort effect\nstarts here',fontsize=9, ha='center', va='bottom')
# plt.colorbar(img1, ax=ax1)


ax2 = fig.add_subplot(gs[0, 1])
ax2.text(-18,-10,'B', fontsize=15)
matrix = DLM_period.hz[40:, 40:]
mask = np.zeros_like(matrix)
mask[np.triu_indices_from(mask)] = True
plot_matrix_mask = np.ma.masked_array(matrix, mask=mask)
# extent = t[0], t[-1], t[-1], t[0]
img2 = ax2.imshow(plot_matrix_mask,cmap='Oranges',vmax=vmax)
ax2.set_xticks(ticks=np.arange(0, t[-1]-t[40]+1, 20), labels=t[40::20], rotation=90)
ax2.set_yticks(ticks=np.arange(0, t[-1]-t[40]+1, 20), labels=t[40::20])
ax2.set_xlabel('Cohort')
ax2.set_ylabel('Time')
ax2.xaxis.set_ticks_position('top')
ax2.xaxis.set_label_position('top')
# plt.colorbar(img2, ax=ax2)
cbar = plt.colorbar(img1,cax=fig.add_axes([0.98, 0.11, 0.02, 0.77]), ticks=ticks)
cbar.ax.get_yaxis().labelpad = 8
cbar.ax.set_ylabel('Hazard rate')
ax2.annotate(text='',xy=(35,36),xytext=(43,36),arrowprops=dict(facecolor='black', width=1.2,headwidth=6,headlength=10))
ax2.text(51.5,36,'Period effect\nstarts here',fontsize=9, ha='center', va='center')
plt.tight_layout()
if export_figs_to_pdf:
    fig.savefig('Fig3.pdf', format='pdf', bbox_inches = "tight")
if export_figs_to_xlsx:
    data = DLM_cohort.hz[40:,40:]
    df = pd.DataFrame(data=data, index=pd.MultiIndex.from_product([t[40:]], names=['time']), columns=t[40:])
    df_to_excel_SI(excel_SI,df,'Figure 3A')
if export_figs_to_xlsx:
    data = DLM_period.hz[40:,40:]
    df = pd.DataFrame(data=data, index=pd.MultiIndex.from_product([t[40:]], names=['time']), columns=t[40:])
    df_to_excel_SI(excel_SI,df,'Figure 3B')